In [ ]:
import json
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

In [ ]:
with open("../output/stats.txt", "r+") as finput:
    j = json.load(finput)

In [ ]:
df = pd.DataFrame.from_dict({x['benchname']: x['benchdata'] for x in j}, orient='index').fillna(0).astype(np.int32)

In [ ]:
df.describe()